In [1]:
import torch
import torchtext
import torch.functional as F
import torch.nn as nn
import spacy
import pandas as pd
import numpy as np
import random
import time
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from spacy.lang.en.stop_words import STOP_WORDS
SEED=42
torch.manual_seed(SEED)
import re
df = pd.read_csv('dataset.csv')

In [2]:
# path = r"C:\Users\win10\Downloads\Datasets\HS Dataset\labeled_data.csv"
# df = pd.read_csv(path)

# subset = {'tweet':[],'label':[]}
# hcount,ocount, ncount =  0,0,0
# for tweet, label in zip(df['tweet'],df['class']):
#     if(label == 0 and hcount<=1400):
#         hcount += 1
#         subset['tweet'].append(tweet)
#         subset['label'].append(label)
#     elif(label == 1 and ocount<=1400):
#         ocount += 1
#         subset['tweet'].append(tweet)
#         subset['label'].append(label)
#     elif(label == 2 and ncount<=1400):
#         ncount+= 1
#         subset['tweet'].append(tweet)
#         subset['label'].append(label)

# dataset =  pd.DataFrame(data = subset, columns = ['tweet','label'])
# dataset.head(10)

# dataset =  dataset.sample(frac=1).reset_index(drop=True)

# dataset.head(10)

# dataset.to_csv('dataset.csv')

In [3]:
class Vocabulary(object):
    """Class to build vocabulary for mapping"""
    def __init__(self, token_to_idx = None, add_unk = True, unk_token = "<UNK>", mask_token="<MASK>", begin_seq_token="<BEGIN>",end_seq_token="<END>"):
        """
        Args:
            token_to_idx: Initialize token to idx dictionary.
            add_unk: Whether to include the unknown token in the vocabulary
            unk_token: How the unknown token is represented in the vocabulary
        """
        
        if token_to_idx is None:
            token_to_idx = {}
            
        self._token_to_idx = token_to_idx
        
        self._idx_to_token = {idx:token 
                             for token, idx in self._token_to_idx.items()}
        
#         self._mask_token = mask_token
        self._unk_token = unk_token
#         self._begin_seq_token = begin_seq_token
#         self._end_seq_token = end_seq_token

#         self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
#         self.begin_seq_index = self.add_token(self._begin_seq_token)
#         self.end_seq_index = self.add_token(self._end_seq_token)
        
        self._add_unk = add_unk
        
    def from_serializable(self):
        """
        Returns:
            A dictionary that can be serialized
        """

        return cls(**contents)
        
    def add_token(self, token):
        """ Update mapping dictionaries given the token
        Args:
            token (str): Token to add to the vocabulary
        Returns:
            index (int): The index corresponding to the token          
        """

        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token

        return index
        
    def lookup_token(self, token):
        """ Retrieves the index associated with the token
            or the UNK index if the token isn't present in the vocabulary

        Args:
            token (str): The token for which the index has to be retrieved
        Returns:
            index (int): The index associated with the token in the dictionary

        Note: 
            'UNK Index' has to be >=0 for the UNK functionality
        """

        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx.get(token)

    def lookup_index(self, index):
        """Retrieve the token associated with the index
        Args:
            index (int): The index to look up
        Returns:
            token (str): The token associated with the index
        Raises:
            KeyError: if the index is not in the vocabulary
        """

        if index not in self._idx_to_token:
            raise KeyError("the index %d is not in the vocabulary" % index)
        else:
            return self._idx_to_token[index]

    def __len__(self):
        """Returns the length of the vocabulary
        """
        return len(self._token_to_idx)      


In [4]:
class Hate_Speech_Tweets(Dataset):
    def __init__(self, df, nlp, vocab):
        """Initializing
        Args:
            df (Pandas DataFrame): Dataframe consisting of tweets and labels
            nlp (spacy object): For preprocessing
            vocab (Vocabulary Object): To vectorize the tweets
        """
        self.df = df
        self.nlp = nlp
        self.vocab = vocab
        measure_len = lambda context: len(context.split(" "))
        self._max_seq_length = max(map(measure_len, self.df.tweet)) + 1
        
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
          
        tweet =  self.df['tweet'].iloc[idx]
        
        label = self.df['label'].iloc[idx]
        return {'tweet':torch.LongTensor(self.preprocess(tweet)), 'label':label}
    
    def preprocess(self, sent):
        
        #Preprocessing and tokenizing
        sent =  re.sub(r'RT','',sent)
        sent =  re.sub('@[a-zA-z0-9_]+','',sent)
        sent =  re.sub(r'http://[^\s<>"]+|www\.[^\s<>"]+','',sent)
        sent =  re.sub(r'&#[0-9]+','',sent)
        sent =  re.sub(r'[0-9]+','',sent)
        sent =  re.sub(r'[^\w\s]', '',sent)
        sent =  sent.strip()
        sent =  " ".join(sent.split())
        sent = [token.lemma_ for token in self.nlp(sent) if token.text not in STOP_WORDS]
        sent = self.vectorize(sent)
        return sent
    
    def vectorize(self, sent):
        """Converts raw text to numeric vectors using the vocabulary
        Args:
            sent (str): The tweet to be vectorized
        Returns:
            vector (list): The vector associated with the tweet
        """
#         vector = [self.vocab.begin_seq_index]
        vector = []
        for token in sent:
            vector.append(vocab.lookup_token(token))
#         vector.append(self.vocab.end_seq_index)

#         out_vector = np.zeros(self._max_seq_length, dtype=np.int64)
#         out_vector[:len(vector)] = vector
#         out_vector[len(vector):] = self.vocab.mask_index
            
        return vector
    

In [5]:
vocab = Vocabulary(token_to_idx=None, add_unk=True)
nlp = spacy.load(name='en_core_web_sm')

def clean(t):
    t = re.sub(r'RT','',t)
    t = re.sub('@[a-zA-z0-9_]+','',t)
    t = re.sub(r'http://[^\s<>"]+|www\.[^\s<>"]+','',t)
    t = re.sub(r'&#[0-9]+','',t)
    t = re.sub(r'[0-9]+','',t)
    t = re.sub(r'[^\w\s]+', '',t)
    t = t.strip()
    t = " ".join(t.split())
    t = [token.lemma_ for token in nlp(t) if token.text not in STOP_WORDS]
    return t

cleaned_tweets = df['tweet'].apply(clean)
text = []

for i in range(len(cleaned_tweets)):
    for word in cleaned_tweets[i]:
        text.append(word)

from collections import Counter
count_dict = Counter(text).most_common(20000)


In [ ]:
for i,word in enumerate(count_dict):
    if(word[1]==1):
        print(word)

In [6]:
for tup in count_dict:
    vocab.add_token(tup[0])

In [7]:
data = Hate_Speech_Tweets(df,nlp,vocab)

In [8]:
# text = []

# for i in range(len(data)):
#     for word in data[i]['tweet']:
#         text.append(word)

# text = " ".join(text)

# x =  ''.join(text)

# plt.figure(figsize = (5,5))
# wc = WordCloud(width=3500, height=3500).generate(x)
# plt.imshow(wc)

In [8]:
vocab.lookup_token('love')

25

In [9]:
vocab.lookup_index(0)

'<UNK>'

In [10]:
def load_glove(filepath):
    """Loads the glove embeddings
    
    Args:
        filepath (str): path to the glove embeddings file
    Return:
        word_to_index (dict): Mappings from word to index
        embeddings (np.array): Embeddings of the words in the vocabulary
    """
    word_to_index = {}
    embeddings = []
    with open(filepath, 'r', encoding='utf-8') as fp:
        for index, line in enumerate(fp):
            # line = word num1 num2 .......
            line = line.split(" ")
            word_to_index[line[0]] = index
            embedding_i = np.array([float(val) for val in line[1:]])
            embeddings.append(embedding_i)
    
    return word_to_index, np.array(embeddings)

In [11]:
def make_embedding_matrix(filepath, words):
    """Create embedding matrix for a specific set of words
    Args:
        word_to_index (dict) : mapping of word to index
        embeddings (list): embeddings of words
        words (list): List of words in the dictionary
    Returns:
        final_embeddings (np..array) : embedding matrix
    """
    
    word_to_idx, embeddings = load_glove(filepath)
    embedding_size = embeddings.shape[1]
    final_embeddings = np.zeros((len(words), embedding_size))
    
    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.zeros(embedding_size)
            
    return final_embeddings

In [12]:
words=[]
for idx in range(0, vocab.__len__()):
    words.append(vocab.lookup_index(idx))

In [13]:
embs = make_embedding_matrix(r"C:\Users\win10\Documents\glove.6B\glove.6B.50d.txt", words)

In [15]:
i=0
words[i], embs[i]

('<UNK>',
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [14]:
i = random.randint(0,4200)
print(i)
data[i]['tweet'], data[i]['label']

2716


(tensor([ 9, 25]), 1)

In [17]:
# train_iterator, test_iterator = BucketIterator.splits(
#     (train_data,test_data),
#     batch_size = 8,
#     sort = False,
#     device = 'cuda')

In [28]:
embs.shape

(3001, 50)

In [39]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, label_size, batch_size, embedding_weights, bidirectional = False):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        self.word_embeddings = nn.Embedding.from_pretrained(embedding_weights, freeze=True, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,
                            bidirectional = bidirectional,batch_first=True)
        if bidirectional:
            self.fc = nn.Linear(hidden_dim*2, label_size)
        else:
            self.fc = nn.Linear(hidden_dim, label_size)
#         self.act = nn.Sigmoid()
 
    def forward(self, sentences, train = True):
        embeds = self.word_embeddings(sentences)
#         packed_embedded = nn.utils.rnn.pack_padded_sequence(embeds, src_len.cpu(), enforce_sorted=False)
        packed_outputs, (hidden,cell) = self.lstm(embeds)
#         hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
#         print(hidden.shape)
#         print(hidden)
        dense_outputs = self.fc(hidden)
        outputs = dense_outputs
        return outputs

In [40]:
BATCH_SIZE = 64
nlabel = 3
hidden_dim = 25
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = LSTMClassifier(embedding_dim=embs.shape[1],hidden_dim=hidden_dim,label_size=nlabel, batch_size=BATCH_SIZE, embedding_weights=torch.from_numpy(embs).float())
model = model.to(device)
 
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
loss_function = nn.CrossEntropyLoss()
 
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [22]:
#Rewrite collate_ FN function, whose input is the sample data of a batch
def collate_fn(batch):
	#Because token_ List is a variable length data, so you need to use a list to load the token of the batch_ list
    token_lists = [item['tweet'] for item in batch]
    #Each label is an int. we take out all the labels in the batch and reassemble them
    labels = [item['label'] for item in batch]
    #Converting labels to tensor
    labels = torch.LongTensor(labels)
    return {
    'token_list': torch.nn.utils.rnn.pad_sequence(token_lists, batch_first=True),
    'label': labels,
    }


#When using dataloader to load data, pay attention to collate_ The FN parameter passes in an overridden function
trainset = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=collate_fn)

In [23]:
for batch in trainset:
    print(batch['token_list'])
    print(batch['label'])
    break

tensor([[ 150,  678,  249,    6, 8015, 8016, 8017, 8018,    0,    0,    0,    0,
            0,    0,    0],
        [6132,  200,  801,   40,    3,  680,  175,  128,  124,   18,    6,  367,
         1776,  864, 6133],
        [ 550, 6576, 6577, 6578, 2647, 2284, 6579,  816, 6580,  323,   46,  589,
            0,    0,    0],
        [   1,    7,  223,    1, 2420,    1,   80,    2,    1, 1265,   28,   23,
            0,    0,    0]])
tensor([0, 0, 2, 1])


In [41]:
from tqdm import tqdm
epochs=15
for epoch in range(epochs):
    time.sleep(1)
    total_loss = 0.0
    total_acc=0.0
    for i, batch in enumerate(tqdm(trainset)):
        feature, label = batch['token_list'].to(device), batch['label'].to(device)
#         batch_length = torch.tensor(33, dtype = torch.int64).unsqueeze(0)
        optimizer.zero_grad()
        output =  model(feature).squeeze()
        loss = loss_function(output, label)
        acc=categorical_accuracy(output,label)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_acc+=acc.item() 
        

    print(f"loss on epoch {epoch+1} = {total_loss/len(trainset)}")
    print(f"accuracy on epoch {epoch+1} = {total_acc/len(trainset)}")

100%|██████████████████████████████████████████████████████████████████████████████| 1051/1051 [00:39<00:00, 26.90it/s]


loss on epoch 1 = 1.0928235091446241
accuracy on epoch 1 = 0.3526799873325736


100%|██████████████████████████████████████████████████████████████████████████████| 1051/1051 [00:33<00:00, 31.36it/s]


loss on epoch 2 = 1.0098866149995807
accuracy on epoch 2 = 0.5013479226322882


100%|██████████████████████████████████████████████████████████████████████████████| 1051/1051 [00:38<00:00, 27.50it/s]


...

loss on epoch 13 = 0.6099386379959241
accuracy on epoch 13 = 0.7611798287345385


100%|██████████████████████████████████████████████████████████████████████████████| 1051/1051 [00:32<00:00, 32.08it/s]


loss on epoch 14 = 0.5972102106283893
accuracy on epoch 14 = 0.7633206470028544


100%|██████████████████████████████████████████████████████████████████████████████| 1051/1051 [00:30<00:00, 34.32it/s]

loss on epoch 15 = 0.5934801458972165
accuracy on epoch 15 = 0.7714874722581041


In [44]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [34]:
def predict_class(model, sent, min_len = 4):
    model.eval()
    sent =  re.sub(r'RT','',sent)
    sent =  re.sub('@[a-zA-z0-9_]+','',sent)
    sent =  re.sub(r'http://[^\s<>"]+|www\.[^\s<>"]+','',sent)
    sent =  re.sub(r'&#[0-9]+','',sent)
    sent =  re.sub(r'[0-9]+','',sent)
    sent =  re.sub(r'[^\w\s]', '',sent)
    sent =  sent.strip()
    sent =  " ".join(sent.split())
    sent = [token.lemma_ for token in nlp(sent) if token.text not in STOP_WORDS]
    vector = []
    for token in sent:
        vector.append(vocab.lookup_token(token))
    tensor = torch.LongTensor(vector)
    tensor = tensor.unsqueeze(0)
    preds = model(tensor.to(device))
#     _, idx = torch.max(m(output[0].reshape(1,3)), dim=1)
    max_preds = preds[0].argmax(dim = 1)
#     print(max_preds)
    return max_preds

In [38]:
predict_class(model=model, sent="My name is noun")

tensor([2], device='cuda:0')

In [46]:
test_sen = 'Not love'
pred_class = predict_class(model, test_sen).item()
itol = {0: 'Hate Speech', 1: 'Offensive', 2: 'Neither'}
print(test_sen)
print(f'Predicted class is: {pred_class} = {itol[int(pred_class)]}.')

Not love
Predicted class is: 2 = Neither.
